<a target="_blank" href="https://colab.research.google.com/github/cohere-ai/notebooks/blob/main/notebooks/Data_Analyst_Agent_Cohere_and_Langchain.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
! pip install --quiet langchain langchain_cohere langchain_experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.2/210.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/14

## Setup
We'll need a Cohere API key here. Grab your key and paste it in the next slide if you have one, or [register](https://dashboard.cohere.ai/welcome/register) and create a new API key.

In [5]:
### LLMs
import os
os.environ['COHERE_API_KEY'] = "XHzIjESLi1QLsV3oZ59TitgswxvNS6V7wsc3peGj"

In [6]:
# Create the Cohere chat model
from langchain_cohere.chat_models import ChatCohere
chat = ChatCohere(model="command-r-plus", temperature=0.3)

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [7]:
import os
from langchain_community.tools.tavily_search import TavilySearchResults

os.environ['TAVILY_API_KEY'] = "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"

internet_search = TavilySearchResults()
internet_search.name = "internet_search"
internet_search.description = "Returns a list of relevant document snippets for a textual query retrieved from the internet."


from langchain_core.pydantic_v1 import BaseModel, Field
class TavilySearchInput(BaseModel):
    query: str = Field(description="Query to search the internet with")
internet_search.args_schema = TavilySearchInput

In [8]:
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL

python_repl = PythonREPL()
repl_tool = Tool(
    name="python_repl",
    description="Executes python code and returns the result. The code runs in a static sandbox without interactive mode, so print output or save output to a file.",
    func=python_repl.run,
)
repl_tool.name = "python_interpreter"

class ToolInput(BaseModel):
    code: str = Field(description="Python code to execute.")
repl_tool.args_schema = ToolInput


In [9]:
from langchain.agents import AgentExecutor
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
from langchain_core.prompts import ChatPromptTemplate

In [10]:

prompt = ChatPromptTemplate.from_template("{input}")



agent = create_cohere_react_agent(
    llm=chat,
    tools=[internet_search, repl_tool],
    prompt=prompt,
)

In [11]:
agent_executor = AgentExecutor(agent=agent, tools=[internet_search, repl_tool], verbose=True)

In [16]:
agent_executor.invoke({
    "input": "Who is Hardik Jindal from IIT Kanpur and in which organisation he is working with in GSOC and display the answers in organised form",
})



> Entering new AgentExecutor chain...

I will search for 'Who is Hardik Jindal from IIT Kanpur' and 'Hardik Jindal GSOC organisation'.
{'tool_name': 'internet_search', 'parameters': {'query': 'Who is Hardik Jindal from IIT Kanpur'}}
[{'url': 'https://hardik1408.github.io/hardik_jindal/about/', 'content': "Hardik Jindal is currently a second-year undergradutae student at the Indian Institute of Technology Kanpur, pursuing a degree in Electrical Engineering. Beyond my academic pursuits, I'm an ardent enthusiast for knowledge, always eager to broaden my horizons and engage in discussions about various ideas."}, {'url': 'https://groups.google.com/g/score-community/c/3yiOf8tleUA', 'content': 'Myself Hardik Jindal, a second year undergraduate in IIT Kanpur. I was going through your project list and found ProjectExplainer highly alligned with my interests. I have had great experience working with LLMs and was part of the Inter IIT Tech meet team for DevRev where I extensively used open LLMs

{'input': 'Who is Hardik Jindal from IIT Kanpur and in which organisation he is working with in GSOC and display the answers in organised form',
 'output': 'Hardik Jindal is a second-year undergraduate student at the Indian Institute of Technology Kanpur, pursuing a degree in Electrical Engineering. He is working with the C2SI (Ceylon Computer Science Institute) organisation for GSOC.',
 'citations': [CohereCitation(start=19, end=30, text='second-year', documents=[{'url': 'https://hardik1408.github.io/hardik_jindal/about/', 'content': "Hardik Jindal is currently a second-year undergradutae student at the Indian Institute of Technology Kanpur, pursuing a degree in Electrical Engineering. Beyond my academic pursuits, I'm an ardent enthusiast for knowledge, always eager to broaden my horizons and engage in discussions about various ideas.", 'id': 'doc_0'}, {'url': 'https://groups.google.com/g/score-community/c/3yiOf8tleUA', 'content': 'Myself Hardik Jindal, a second year undergraduate in 

In [14]:
agent_executor.invoke({
    "input": "Create a plot of the number of full time employees at the 3 tech companies with the highest market cap in the United States in 2024.",
})



> Entering new AgentExecutor chain...

First, I will search for the 3 tech companies with the highest market cap in the United States in 2024. Then, I will search for the number of full-time employees at each of these companies. Finally, I will use Python to create a plot of the number of full-time employees at these companies.
{'tool_name': 'internet_search', 'parameters': {'query': '3 tech companies with highest market cap in the United States 2024'}}
[{'url': 'https://disfold.com/united-states/sector/technology/companies/', 'content': 'Unlock Financial AI:\nStart your Free Trial of\nDisfold AI, Now!\nTop Technology Companies from the United States as of Jan. 01, 2024\n(Dec. 1, 2023)\nTop Technology Companies from the United States as of Jan. 01, 2024\n1.\nApple\nMarket Cap (USD):\n$2.866 T\nStock:\nAAPL\nwb_incandescent\nCountry:\nUnited States\nSector:\nTechnology\nIndustry:\nConsumer Electronics\n2.\nMicrosoft\nMarket Cap (USD):\n$2.755 T\nStock:\nMSFT\nwb_incandescent\nCountry:

{'input': 'Create a plot of the number of full time employees at the 3 tech companies with the highest market cap in the United States in 2024.',
 'output': "Here is a plot of the number of full-time employees at the 3 tech companies with the highest market cap in the United States as of January 1, 2024:\n\nApple - 161,000\nMicrosoft - 228,000\nNvidia - 29,600\n\n![Number of Full-Time Employees at Top 3 Tech Companies in the US by Market Cap]('number_of_full_time_employees_at_top_3_tech_companies_in_the_us_by_market_cap.png')",
 'citations': [CohereCitation(start=131, end=146, text='January 1, 2024', documents=[{'url': 'https://disfold.com/united-states/sector/technology/companies/', 'content': 'Unlock Financial AI:\nStart your Free Trial of\nDisfold AI, Now!\nTop Technology Companies from the United States as of Jan. 01, 2024\n(Dec. 1, 2023)\nTop Technology Companies from the United States as of Jan. 01, 2024\n1.\nApple\nMarket Cap (USD):\n$2.866 T\nStock:\nAAPL\nwb_incandescent\nCoun

In [13]:
agent_executor.invoke({
    "input": "Hey how are you?",
})



> Entering new AgentExecutor chain...

I will respond to the user's greeting.
{'tool_name': 'directly_answer', 'parameters': {}}
directly_answer is not a valid tool, try one of [internet_search, python_interpreter].Relevant Documents: None
Cited Documents: None
Answer: Hello! I'm an AI assistant, so I don't have feelings as such, but I'm here to help you with your queries. How can I assist today?
Grounded answer: Hello! I'm an AI assistant, so I don't have feelings as such, but I'm here to help you with your queries. How can I assist today?

> Finished chain.


{'input': 'Hey how are you?',
 'output': "Hello! I'm an AI assistant, so I don't have feelings as such, but I'm here to help you with your queries. How can I assist today?",
 'citations': []}